In [ ]:
import pandas as pd
import glob
import os
from difflib import get_close_matches

# 📁 디렉토리 설정
csv_dir = '../nodate'
refined_dir = '../정제data'
os.makedirs(refined_dir, exist_ok=True)

# 🗑️ 기존 정제 파일 삭제
refined_files = glob.glob(os.path.join(csv_dir, '*_정제.csv')) + glob.glob(os.path.join(refined_dir, '*_정제.csv'))
for file_path in refined_files:
    try:
        os.remove(file_path)
        print(f"🗑️ 삭제 완료: {file_path}")
    except Exception as e:
        print(f"⚠️ 삭제 실패: {file_path} → {e}")
print(refined_files)
# 📄 CSV 파일 목록 (정제 전, "_정제.csv" 제외)
files = [f for f in glob.glob(os.path.join(csv_dir, '*.csv')) if '_정제.csv' not in f]
files.sort(reverse=True)
# 📌 2023년 기준 역명 확보
base_file = next((f for f in files if '2023년' in f), None)
if base_file is None:
    raise ValueError("❌ 2023년 기준 파일이 없습니다. 파일명에 '2023'이 포함되어야 합니다.")

base_df = pd.read_csv(base_file, encoding='cp949')
standard_names = set(base_df['역명'].unique())

print(f"✅ 기준 파일: {os.path.basename(base_file)}")
print(f"✅ 기준 역 개수: {len(standard_names)}")

# ✅ 이름 매핑 테이블 생성
name_mapping = {}
for file in files:
    df = pd.read_csv(file, encoding='cp949')
    for name in df['역명'].unique():
        if name in standard_names:
            name_mapping[name] = name
        elif name not in name_mapping:
            match = get_close_matches(name, standard_names, n=1, cutoff=0.9)
            name_mapping[name] = match[0] if match else None  # None이면 삭제 대상
# 기준 역번호 → 역명 매핑
standard_map = base_df[['역번호', '역명']].drop_duplicates().set_index('역번호')['역명'].to_dict()

# 이름 매핑 처리
change_log = []
delete_log = []

for file in files:
    df = pd.read_csv(file, encoding='cp949', low_memory=False, dtype={'역번호': str})
    original_names = df['역명'].copy()
    
    # 매핑: 역번호 기준으로 기준 역명으로 통일
    df['역명'] = df['역번호'].map(standard_map)

    # 삭제 대상: 기준에 없는 역번호 (즉, map 결과가 NaN인 경우)
    delete_rows = df[df['역명'].isna()]
    df = df[df['역명'].notna()].reset_index(drop=True)

    # 로그 기록 - 변경
    for i, (old, new) in enumerate(zip(original_names, df['역명'])):
        if pd.isna(new):
            continue
        if old != new:
            change_log.append({
                '파일명': os.path.basename(file),
                '행번호': i,
                '기존 역명': old,
                '변경 후 역명': new
            })

    # 로그 기록 - 삭제
    for i, row in delete_rows.iterrows():
        delete_log.append({
            '파일명': os.path.basename(file),
            '행번호': i,
            '삭제된 역번호': row['역번호'],
            '삭제된 역명': row['역명']
        })

    # 저장
    new_filename = os.path.splitext(os.path.basename(file))[0] + "_정제.csv"
    save_path = os.path.join(refined_dir, new_filename)
    df.to_csv(save_path, index=False, encoding='utf-8-sig')
    print(f"✅ 저장 완료: {save_path}")
